In [7]:
# Import libraries and api keys
import hvplot.pandas
import pandas as pd
import requests

from api_keys import geoapify_key

In [8]:
# Load CSV file created in WeatherPy
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,collie,-33.3667,116.1500,32.27,35,0,2.69,AU,1733967209
1,1,blackmans bay,-43.0167,147.3167,18.18,61,100,0.89,AU,1733967210
2,2,lompoc,34.6391,-120.4579,14.45,63,0,2.57,US,1733967211
3,3,san patricio,28.0170,-97.5169,10.97,36,0,3.91,US,1733967212
4,4,lihue,21.9789,-159.3672,25.77,73,75,7.72,US,1733967213


In [9]:
# Plot cities on the holoview plot

map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [10]:
# Narrow down cities that fit criteria 

narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0),:
]

# Drop null values
narrowed_city_df = narrowed_city_df.dropna()

narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,margaret river,-33.9500,115.0667,26.25,55,0,3.69,AU,1733967007
58,58,montevideo,-34.8335,-56.1674,21.84,64,0,3.60,UY,1733967102


In [11]:
# Figure out hotel arrangements

hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name"
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
24,margaret river,AU,-33.9500,115.0667,55,
58,montevideo,UY,-34.8335,-56.1674,64,


In [12]:
# Find the first hotel located within 10,000 metres 

radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

print("Starting hotel search")

# Iterate through hotel_df
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Build parameters
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make API request
    name_address = requests.get(base_url, params=params)
    print(name_address.status_code)
    
    # JSONify
    name_address = name_address.json()
    
    # Grab the first hotel from results
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
# Display data
hotel_df

Starting hotel search
200
200


,City,Country,Lat,Lng,Humidity,Hotel Name
24,margaret river,AU,-33.9500,115.0667,55,Margaret River Hotel
58,montevideo,UY,-34.8335,-56.1674,64,Hotel Escuela


In [21]:
# Plot holoview and show hotels on the map

map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

## Insights

**1. Geographical Distribution:**

The map shows two points:
Margaret River (Western Australia, near latitude -35, longitude ~115).
Montevideo (South America, near latitude -35, longitude ~-56).

Both locations are in the Southern Hemisphere and within similar latitudinal ranges, possibly suggesting a comparison of cities with similar climates or geographical conditions.

**2. Interactive Features:**

The map includes interactive elements, such as hover data (city names) and zoom/pan controls, which make it user-friendly and exploratory.
The legend at the bottom-right identifies the cities by their colors, providing an easy reference for users.

**3. Climatic/Environmental Insights:**

If the point size represents humidity, we observe that Montevideo has a larger circle than Margaret River, suggesting Montevideo may experience higher humidity levels.
This visualization could highlight climatic differences or similarities between coastal regions across continents.
